In [1]:
import torchvision,torch,time
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from torch.nn import Conv2d, MaxPool2d, Flatten, Linear, Sequential

# 设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("using {} device.".format(device))

dataset_transform = torchvision.transforms.ToTensor()
train_data = torchvision.datasets.FashionMNIST(root="../dataset", train=True, transform=dataset_transform, download=False)
test_data = torchvision.datasets.FashionMNIST(root="../dataset", train=False, transform=dataset_transform, download=False)
train_dataloader = DataLoader(train_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)
# length 长度
train_data_size = len(train_data)
test_data_size = len(test_data)
print("训练数据集的长度为：{}".format(train_data_size))
print("测试数据集的长度为：{}".format(test_data_size))

using cuda device.
训练数据集的长度为：60000
测试数据集的长度为：10000


In [2]:
# 创建网络模型    
class Tudui(nn.Module):
    def __init__(self):
        super(Tudui, self).__init__()
        self.model1 = Sequential(
            Conv2d(1, 32, 5, padding=4),
            MaxPool2d(2),
            Conv2d(32, 32, 5, padding=2),
            MaxPool2d(2),
            Conv2d(32, 64, 5, padding=2),
            MaxPool2d(2),
            Flatten(),
            Linear(1024, 64),
            Linear(64, 10)
        )

    def forward(self, x):
        x = self.model1(x)
        return x
    
    
tudui = Tudui()
tudui = tudui.to(device)
loss_fn = nn.CrossEntropyLoss() # 交叉熵损失函数
loss_fn = loss_fn.to(device)
learning_rate = 1e-2
optimizer = torch.optim.SGD(tudui.parameters(), lr=learning_rate)

total_train_step = 0    # 训练次数
total_test_step = 0     # 测试次数
epoch = 10              # 训练轮数
writer = SummaryWriter("../logs/hello_train")

In [3]:
last_time = time.time()
for i in range(epoch):
    print("-------第 {} 轮训练开始-------".format(i+1))
    # 训练
    tudui.train() # 对某些特定的层需要此句
    for data in train_dataloader:
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = tudui(inputs)
        loss = loss_fn(outputs, labels)

        # 优化器优化模型
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_train_step = total_train_step + 1
        if total_train_step % 100 == 0:
            print("训练次数：{}, Loss: {}".format(total_train_step, loss.item()))
            writer.add_scalar("train_loss", loss.item(), total_train_step)

    # 测试
    tudui.eval() # 对某些特定的层需要此句
    total_test_loss = 0
    total_accuracy = 0
    with torch.no_grad():
        for data in test_dataloader:
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = tudui(inputs)
            loss = loss_fn(outputs, labels)
            total_test_loss = total_test_loss + loss.item()
            accuracy = (outputs.argmax(1) == labels).sum() # argmax(0)竖着看，argmax(1)横着看
            total_accuracy = total_accuracy + accuracy

    print("整体测试集上的Loss: {}".format(total_test_loss))
    print("整体测试集上的正确率: {}".format(total_accuracy/test_data_size))
    writer.add_scalar("test_loss", total_test_loss, total_test_step)
    writer.add_scalar("test_accuracy", total_accuracy/test_data_size, total_test_step)
    total_test_step = total_test_step + 1

    # torch.save(tudui, "../models/tudui_{}.pth".format(i))
    torch.save(tudui.state_dict(), "../models/hello/tudui_{}.pth".format(i))
    delta_time = time.time() - last_time
    last_time = time.time()
    print("模型已保存，耗时：{}".format(delta_time))

writer.close()

-------第 1 轮训练开始-------
训练次数：100, Loss: 2.2272961139678955
训练次数：200, Loss: 1.6167548894882202
训练次数：300, Loss: 0.8766621351242065
训练次数：400, Loss: 1.1124264001846313
训练次数：500, Loss: 0.5536853075027466
训练次数：600, Loss: 0.7266916036605835
训练次数：700, Loss: 0.7898034453392029
训练次数：800, Loss: 0.8246686458587646
训练次数：900, Loss: 0.6201309561729431
整体测试集上的Loss: 110.34182271361351
整体测试集上的正确率: 0.7374999523162842
模型已保存，耗时：7.039867162704468
-------第 2 轮训练开始-------
训练次数：1000, Loss: 0.9773823618888855
训练次数：1100, Loss: 0.49233925342559814
训练次数：1200, Loss: 0.7686746716499329
训练次数：1300, Loss: 0.6144752502441406
训练次数：1400, Loss: 0.7183102369308472
训练次数：1500, Loss: 0.44043901562690735
训练次数：1600, Loss: 0.6567753553390503
训练次数：1700, Loss: 0.8016732931137085
训练次数：1800, Loss: 0.4023885726928711
整体测试集上的Loss: 93.99767392873764
整体测试集上的正确率: 0.781499981880188
模型已保存，耗时：6.652388572692871
-------第 3 轮训练开始-------
训练次数：1900, Loss: 0.5995471477508545
训练次数：2000, Loss: 0.74496990442276
训练次数：2100, Loss: 0.549069881439209
训练次数